In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.integrate import solve_ivp
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Define double pendulum dynamics
def double_pendulum(t, y, l1, l2, m1, m2, g):
    theta1, z1, theta2, z2 = y
    delta = theta2 - theta1
    denom1 = (m1 + m2) * l1 - m2 * l1 * np.cos(delta) ** 2
    denom2 = (l2 / l1) * denom1

    dydt = np.zeros_like(y)
    dydt[0] = z1
    dydt[1] = (
        (m2 * l1 * z1 ** 2 * np.sin(delta) * np.cos(delta)
         + m2 * g * np.sin(theta2) * np.cos(delta)
         + m2 * l2 * z2 ** 2 * np.sin(delta)
         - (m1 + m2) * g * np.sin(theta1))
        / denom1
    )
    dydt[2] = z2
    dydt[3] = (
        (-m2 * l2 * z2 ** 2 * np.sin(delta) * np.cos(delta)
         + (m1 + m2) * g * np.sin(theta1) * np.cos(delta)
         - (m1 + m2) * l1 * z1 ** 2 * np.sin(delta)
         - (m1 + m2) * g * np.sin(theta2))
        / denom2
    )
    return dydt

In [ ]:
# Generate data
n_pendulums = 5000
train_data, val_data, test_data = [], [], []

In [ ]:
for _ in range(n_pendulums):
    l1, l2 = np.random.uniform(0.5, 2.0, 2)
    m1, m2 = np.random.uniform(0.5, 2.0, 2)
    g = 9.81
    y0 = np.random.uniform(-np.pi, np.pi, 4)
    t_span = (0, 10)
    t_eval = np.linspace(t_span[0], t_span[1], 200)
    sol = solve_ivp(double_pendulum, t_span, y0, t_eval=t_eval, args=(l1, l2, m1, m2, g))
    theta1, theta1_dot, theta2, theta2_dot = sol.y
    X = np.vstack([theta1, theta2, theta1_dot, theta2_dot]).T
    train_data.append((X, l1, l2, m1, m2, g))

In [ ]:
# Visualize examples from the dataset
def visualize_example(example):
    X, l1, l2, _, _, _ = example
    theta1, theta2 = X[:, 0], X[:, 1]

    # Compute positions
    x1 = l1 * np.sin(theta1)
    y1 = -l1 * np.cos(theta1)
    x2 = x1 + l2 * np.sin(theta2)
    y2 = y1 - l2 * np.cos(theta2)

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(-2.5, 2.5)
    ax.set_ylim(-2.5, 2.5)
    ax.set_aspect('equal')
    line, = ax.plot([], [], 'o-', lw=2)

    def update(frame):
        line.set_data([0, x1[frame], x2[frame]], [0, y1[frame], y2[frame]])
        return line,

    ani = FuncAnimation(fig, update, frames=len(theta1), blit=True, interval=50)
    plt.show()

In [ ]:
# Visualize first 3 examples
for i in range(3):
    visualize_example(train_data[i])

In [ ]:
# Normalize data
scaler = StandardScaler()
X_train = np.vstack([example[0] for example in train_data])
scaler.fit(X_train)
train_data = [(scaler.transform(example[0]), *example[1:]) for example in train_data]

In [ ]:
# Save dataset
with open('double_pendulum_dataset.pkl', 'wb') as f:
    pickle.dump({'train': train_data}, f)

In [ ]:
# Define Transformer-based model
class PhysicsInformedTransformer(nn.Module):
    def __init__(self, input_dim, output_dim, d_model=128, nhead=4, num_layers=3):
        super(PhysicsInformedTransformer, self).__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead), num_layers=num_layers
        )
        self.decoder = nn.Linear(d_model, output_dim)
        self.input_projection = nn.Linear(input_dim, d_model)
        self.activation = nn.SiLU()

    def forward(self, x):
        x = self.input_projection(x)
        x = self.activation(x)
        x = x.permute(1, 0, 2)
        x = self.encoder(x)
        x = x.permute(1, 0, 2)
        x = self.decoder(x)
        return x

In [ ]:
# Physics-informed loss
def physics_loss(y_pred, y_true, params_batch):
    l1, l2, m1, m2, g = params_batch.T
    theta1, theta2, theta1_dot, theta2_dot = torch.split(y_true, 1, dim=2)
    theta1_ddot, theta2_ddot = torch.split(y_pred, 1, dim=2)

    delta = theta2 - theta1
    physics_term1 = (
        (m2 * l1 * theta1_dot ** 2 * torch.sin(delta) * torch.cos(delta)
         + m2 * g * torch.sin(theta2) * torch.cos(delta)
         + m2 * l2 * theta2_dot ** 2 * torch.sin(delta)
         - (m1 + m2) * g * torch.sin(theta1))
        / ((m1 + m2) * l1 - m2 * l1 * torch.cos(delta) ** 2)
    )
    physics_term2 = (
        (-m2 * l2 * theta2_dot ** 2 * torch.sin(delta) * torch.cos(delta)
         + (m1 + m2) * g * torch.sin(theta1) * torch.cos(delta)
         - (m1 + m2) * l1 * theta1_dot ** 2 * torch.sin(delta)
         - (m1 + m2) * g * torch.sin(theta2))
        / ((l2 / l1) * ((m1 + m2) * l1 - m2 * l1 * torch.cos(delta) ** 2))
    )

    loss = torch.mean((theta1_ddot - physics_term1) ** 2 + (theta2_ddot - physics_term2) ** 2)
    return loss

In [ ]:
# Load dataset
with open('double_pendulum_dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)
train_data = dataset['train']

train_loader = DataLoader(
    TensorDataset(
        torch.tensor(np.vstack([example[0] for example in train_data]), dtype=torch.float32),
        torch.tensor(np.vstack([example[0][:, 2:] for example in train_data]), dtype=torch.float32),
        torch.tensor(np.array([example[1:] for example in train_data]), dtype=torch.float32)
    ),
    batch_size=64, shuffle=True
)

In [ ]:
# Model training
model = PhysicsInformedTransformer(input_dim=4, output_dim=2)
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

num_epochs = 100
patience = 10
best_val_loss = float('inf')
patience_counter = 0

In [ ]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for x_batch, y_batch, params_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_batch) + 0.1 * physics_loss(y_pred, y_batch, params_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    print(f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}")

In [ ]:
# Save the model
torch.save(model.state_dict(), 'physics_informed_transformer.pth')

In [ ]:
# Visualization
def visualize_prediction(example, model, scaler):
    X, l1, l2, _, _, _ = example
    theta1, theta2 = X[:, 0], X[:, 1]

    X_tensor = torch.tensor(scaler.transform(X), dtype=torch.float32)
    y_pred = model(X_tensor.unsqueeze(0)).detach().numpy()[0]

    theta1_pred, theta2_pred = y_pred[:, 0], y_pred[:, 1]

    x1 = l1 * np.sin(theta1)
    y1 = -l1 * np.cos(theta1)
    x2 = x1 + l2 * np.sin(theta2)
    y2 = y1 - l2 * np.cos(theta2)

    x1_pred = l1 * np.sin(theta1_pred)
    y1_pred = -l1 * np.cos(theta1_pred)
    x2_pred = x1_pred + l2 * np.sin(theta2_pred)
    y2_pred = y1_pred - l2 * np.cos(theta2_pred)

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(-2.5, 2.5)
    ax.set_ylim(-2.5, 2.5)
    ax.set_aspect('equal')
    line_true, = ax.plot([], [], 'o-', lw=2, label='Ground Truth')
    line_pred, = ax.plot([], [], 'o--', lw=2, label='Prediction')
    ax.legend()

    def update(frame):
        line_true.set_data([0, x1[frame], x2[frame]], [0, y1[frame], y2[frame]])
        line_pred.set_data([0, x1_pred[frame], x2_pred[frame]], [0, y1_pred[frame], y2_pred[frame]])
        return line_true, line_pred

    ani = FuncAnimation(fig, update, frames=len(theta1), blit=True, interval=50)
    plt.show()


In [ ]:
# Test visualization
visualize_prediction(train_data[0], model, scaler)